In [11]:
from configparser import ConfigParser

config = ConfigParser()
config.read('telegram.ini')

api_id = config['api']['id']
api_hash = config['api']['hash']

phone = config['user']['phone']
password = config['user']['password']
session = config['user']['session']

In [ ]:
import os.path
import datetime
import logging

from telethon import TelegramClient, functions
from telethon.tl.types import Chat, Channel, User

try:
    await client.disconnect()
except:
    logging.exception('disconnect:')
client = TelegramClient(session, api_id, api_hash, use_ipv6=True)
await client.start(phone, password)

In [13]:
dialogs = await client.get_dialogs(ignore_migrated=True)
print('dialogs:', len(dialogs))

groups = [d for d in dialogs if type(d.entity) != User]
groups = list(sorted(groups, key = lambda g: g.date))
print('groups:', len(groups))

users = [d for d in dialogs if type(d.entity) == User]
users = list(sorted(users, key = lambda g: g.date))
print('users:', len(users))

dialogs: 613
groups: 212
users: 401


In [14]:
old_users = [u for u in users if (datetime.datetime.now() - u.date.replace(tzinfo=None) > datetime.timedelta(days = 31 * 6))]
print(len(old_users))

12


In [184]:
WHITELISTED = [
]

def print_group(g):
    if g.name in WHITELISTED:
        return
    print(g.name, "| ", g.date, "| ", repr(g.message.message))
        
async def drop_group(g):
    if g.name in WHITELISTED:
        return
    try:
        await client.delete_dialog(g)
        await client(functions.messages.DeleteHistoryRequest(
            peer=g.entity,
            max_id=0,
            just_clear=True,
            revoke=False,
            min_date=datetime.datetime.now() - datetime.timedelta(days=365*10),
            max_date=datetime.datetime.now()
        ))
    except:
        logging.exception('drop:')
        
for g in old_users[:]:
    print_group(g)
#     await drop_group(g)